In [1]:
import torch
from torchvision import transforms
from PIL import Image
import os
import pandas as pd
import json

C:\Users\jerzy_p7rtwpj\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\jerzy_p7rtwpj\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [91]:
def read_in_data(dir_str, ref_json):
    print(dir_str)
    print(ref_json.keys())
    for key in ref_json.keys():
        print(key)
        print(ref_json[key][0])
    #print(ref_json['box'][0])
    print(ref_json)
    id_to_img = {}
    df = pd.DataFrame(columns=['id', 'image_id', 'bbox', 'category_id', 'img'])

    for line in ref_json["images"]:
        filename = line["file_name"]
        img = Image.open(dir_str + "/" + filename)
        id_to_img[line["id"]] = img

    for box in ref_json["annotations"]:
        box_coords = box["bbox"]
        # im1 = im.crop((left, top, right, bottom))
        # [lewy górny róg, szerokość, wysokość]
        box_coords = [box_coords[0], box_coords[1], box_coords[0] + box_coords[2], box_coords[1] + box_coords[3]]
        
        image = id_to_img[box["image_id"]].crop(box_coords)
        if len(image.getbands()) > 3:
            image_ = Image.new("RGB", image.size, (255, 255, 255))
            image_.paste(image, mask=image.split()[3]) 
        else:
            image_ = image

        box["img"] = image_
        df.loc[box["id"]] = box


    df.set_index("id")
    return df

In [125]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)

dir_str = "../public_dataset/reference_images_part1"
df = read_in_data(dir_str, ref_json)
df.head()



../public_dataset/reference_images_part1
dict_keys(['images', 'annotations', 'categories'])
images
{'id': 4, 'width': 1280, 'height': 720, 'file_name': 'nomagic_mulled_wine_3.png'}
annotations
{'id': 12, 'image_id': 4, 'bbox': [665, 448, 206, 174], 'category_id': 8}
categories
{'id': 1, 'name': 'nomagic_brush'}
{'images': [{'id': 4, 'width': 1280, 'height': 720, 'file_name': 'nomagic_mulled_wine_3.png'}, {'id': 5, 'width': 1280, 'height': 720, 'file_name': 'nomagic_samsung_1.png'}, {'id': 6, 'width': 1280, 'height': 720, 'file_name': 'nomagic_brush_2.png'}, {'id': 7, 'width': 1280, 'height': 720, 'file_name': 'nomagic_cat_eye_1.png'}, {'id': 9, 'width': 1280, 'height': 720, 'file_name': 'nomagic_cat_eye_2.png'}, {'id': 11, 'width': 1280, 'height': 720, 'file_name': 'nomagic_boxers_2.png'}, {'id': 12, 'width': 1280, 'height': 720, 'file_name': 'nomagic_graphics_cards_4.png'}, {'id': 13, 'width': 1280, 'height': 720, 'file_name': 'nomagic_graphics_cards_3.png'}, {'id': 14, 'width': 1280,

,id,image_id,bbox,category_id,img
12,12,4,"[665, 448, 206, 174]",8,<PIL.Image.Image image mode=RGB size=206x174 a...
13,13,4,"[475, 186, 236, 186]",8,<PIL.Image.Image image mode=RGB size=236x186 a...
14,14,4,"[962, 76, 229, 217]",8,<PIL.Image.Image image mode=RGB size=229x217 a...
15,15,4,"[778, 250, 163, 135]",8,<PIL.Image.Image image mode=RGB size=163x135 a...
16,16,4,"[184, 377, 244, 194]",8,<PIL.Image.Image image mode=RGB size=244x194 a...


In [93]:
def read_in_data_reverse(df, ref_json):
    images = pd.DataFrame()
    images = images.assign(id = df['id'])
    images = images.assign(id = df['category_id'])
    print(images)
    images_dict = (images).to_json(orient='records')
    print(images_dict[0])
    for line in images_dict:
        print(line['id'])
        #print(ref_json["images"]['id' == 4 ])
        

    #print(data_dict)
#     df.set_index("id")
    return df
read_in_data_reverse(df, ref_json)

      id
12    12
13    13
14    14
15    15
16    16
..   ...
277  277
278  278
279  279
280  280
294  294

[144 rows x 1 columns]
[


TypeError: string indices must be integers

In [88]:
dataList = [{'a': 1}, {'a': 3, 'b': 20}, {'a': 5}]
for index in range(len(dataList)):
    print(dataList[index]['a'])

1
3
5


In [109]:
with open("../public_dataset/images_part1_test_public.json") as f:
    ref_json = json.load(f)
    #print(ref_json)
    for key in ref_json.keys():
        print(key)
        print(ref_json[key][0])

images
{'id': 156, 'width': 1280, 'height': 720, 'file_name': '20220221-212932.557424-scout.png', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
annotations
{'id': 2987, 'image_id': 156, 'category_id': None, 'area': 8859, 'bbox': [282.21, 253.88, 59.6, 169.87], 'iscrowd': 0, 'occluded': False, 'attributes': {'occluded': ['Not-Occluded']}, 'group_id': 0}
licenses
{'name': '', 'id': 0, 'url': ''}
info


KeyError: 0

In [141]:
with open("../public_dataset/reference_images_part1.json") as f:
    ref_json = json.load(f)
def insert_predictions_to_json(df, ref_json):
    final = pd.DataFrame()
    final = final.assign(id = df['id'])
    final = final.assign(category_id = df['category_id'])
    anno = ref_json['annotations']
    for index in range(len(anno)):
        anno_id = anno[index]['id']
        cat_id = final[final.id==anno_id]['category_id'].values[0]
        anno[index]['category_id'] = cat_id
    ref_json['annotations'] = anno
    return ref_json

[{'id': 12, 'image_id': 4, 'bbox': [665, 448, 206, 174], 'category_id': 8}, {'id': 13, 'image_id': 4, 'bbox': [475, 186, 236, 186], 'category_id': 8}, {'id': 14, 'image_id': 4, 'bbox': [962, 76, 229, 217], 'category_id': 8}, {'id': 15, 'image_id': 4, 'bbox': [778, 250, 163, 135], 'category_id': 8}, {'id': 16, 'image_id': 4, 'bbox': [184, 377, 244, 194], 'category_id': 8}, {'id': 17, 'image_id': 4, 'bbox': [960, 422, 238, 228], 'category_id': 8}, {'id': 18, 'image_id': 4, 'bbox': [446, 437, 169, 155], 'category_id': 8}, {'id': 19, 'image_id': 4, 'bbox': [188, 136, 245, 183], 'category_id': 8}, {'id': 20, 'image_id': 5, 'bbox': [718, 152, 331, 187], 'category_id': 9}, {'id': 21, 'image_id': 5, 'bbox': [215, 116, 146, 262], 'category_id': 9}, {'id': 22, 'image_id': 5, 'bbox': [206, 409, 272, 236], 'category_id': 9}, {'id': 23, 'image_id': 5, 'bbox': [811, 265, 306, 233], 'category_id': 9}, {'id': 24, 'image_id': 5, 'bbox': [452, 119, 253, 244], 'category_id': 9}, {'id': 25, 'image_id': 5,